# Short term Load Forcasting (Regression Machine Learning)

You have been provided with **Load Forecasting** data set as a single file, `dataset.csv`. We obtained it at http://www.mathworks.com/videos/electricity-load-and-price-forecasting-with-matlab-81765.html. For some background information you can also watch the video tutorial given in the link above:

Before you start on this notebook, copy the datasetset `dataset.csv` in the same directory.

### 1. Set up notebook and load dataset

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn import linear_model  # linear regression library
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

This next snippet of code loads the load forecasting dataset. There are 50000 data points, each with 8 predictor variables that are input `x` and one response variable (which we'll denote `y`).

Make sure the file `'dataset.csv'` is in the same directory as this notebook.

In [4]:
dataset = pd.read_csv('dataset.csv') # Reads the dataset in pandas dataframe

features = ['DryBulb', 'DewPoint', 'Hour', 'Weekday', 
            'IsWorkingDay', 'P_W_S_hour_l', 'P_D_S_hour_l', 'prev24HrAveLoad']
dataset.head() # This command tell the first 5 entries of the dataset

,DryBulb,DewPoint,Hour,Weekday,IsWorkingDay,P_W_S_hour_l,P_D_S_hour_l,prev24HrAveLoad,target_variable
0,37,25,1,5,0,12230,12230,509.583333,12230
1,37,25,2,5,0,11534,11534,990.166667,11534
2,39,24,3,5,0,11038,11038,1450.083333,11038
3,38,22,4,5,0,10777,10777,1899.125000,10777
4,37,20,5,5,0,10764,10764,2347.625000,10764


The dataset contains 8 input variable which contains the contextual information and historical load and temperature data. The variable includes:
- DryBulb temperature
- DewPoints
- Hours (data is collected in hourly samples so each day has 24 values)
- Weekday number [1 for Monday and 7 Sunday]
- Holyday flag [IsWorkingDay 0 for working 1 for holiday]
- prvious week same hour load (P_W_S_hour_l) 168 lagging entry i.e. 168 hours per week
- prevuous day same hour load (P_D_S_hour_l) 24 lagging entry
- prev24HrAveLoad (Average load for the perticular day)

Target variable is actual load at that perticular hour that is need to be predicted

In [5]:
x = np.array(dataset.drop('target_variable', axis=1))  # predictors
y = np.array(dataset['target_variable'])               # response variable

## Q1. Predict `y` without using `x`. Please calculate prediction and MSE using y alone?

If we want to predict `y` without knowledge of `x`, what value would be predict? The <font color="magenta">mean</font> value of `y`.

In this case, the mean squared error (MSE) associated with the prediction is simply the variance of `y`. Please, not the For every real value $y_i$ we have predicted one $y^i$. The the absoulut value of the difference is the error for data point $i$. This is denoted by $e_i$ and given by formula

$e_i$=|$y_i$−$\hat{y_i}$|. We can take an average of these pointwise errors:

$\frac{1}{N}\sum_i^N \left| y - \hat{y}_i\right|.$

This error is called *Mean Absolute Error*.

However, for various reasons in many situations we prefer to square the distance before taking the sum. This is called Mean Squared Error and we denote it by MSE. So, 

$MSE(\hat{Y}) = \frac{1}{N}\sum_i^N \left(y_i - \hat{y}_i\right)^2$

$MAPE = \frac{1}{N}\sum_i^N \lvert\frac{y_i - \hat{y}_i}{y_i}\rvert$

## Q2. Predict `y` using a single feature of `x`

Here we define a function, **one_feature_regression**, that takes `x` and `y`, along with the index `f` of a single feature and fits a linear regressor to `(x[f],y)`. It then plots the data along with the resulting line.

In [6]:
def one_feature_regression(x,y,f):
    if (f < 0) or (f > 7):
        print("Feature index is out of bounds")
        return
    regr = linear_model.LinearRegression()
    x1 = x[:,[f]]
    regr.fit(x1, y)
    # Make predictions using the model
    y_pred = regr.predict(x1)
    # Plot data points as well as predictions
    plt.plot(x1, y, 'bo')
    plt.plot(x1, y_pred, 'r-', linewidth=3)
    plt.xlabel(features[f], fontsize=14)
    plt.ylabel('Load Progression', fontsize=14)
    plt.show()
    print("Mean squared error: ", mean_squared_error(y, y_pred))
    print('Mean Absolute Percentage Errror', mean_absolute_percentage_error(y, y_pred)*100)
    return regr

### (a) You have been given one_feature_regression function. Please use th function to develop a regression model using feature #7 (P_D_S_hour_l), and find out MSE, w, and b? (b) Please, identify <font color="magenta"> second best feature </font> with the lowest MSE?

## Q3. Predict `y` using a specified subset of features from `x`

The function **feature_subset_regression** is just like **one_feature_regression**, but this time uses a list of features `flist`.

In [7]:
def feature_subset_regression(x,y,flist):
    if len(flist) < 1:
        print("Need at least one feature")
        return
    for f in flist:
        if (f < 0) or (f > 7):
            print("Feature index is out of bounds")
            return
    regr = linear_model.LinearRegression()
    x1 = x[:,flist]
    regr.fit(x1, y)
    # Make predictions using the model
    y_pred = regr.predict(x1)

    print("Mean squared error: ", mean_squared_error(y, y_pred))
    print('Mean Absolute Percentage Errror', mean_absolute_percentage_error(y, y_pred)*100)
    return regr

### (a) Using the *feature_subset_regression*, calculate MSE, w and b usng features #7 (P_D_S_hour_l) and #8 (prev24HrAveLoad). (b) Calculate MSE and MAPE using all 8 features?

# Finally, use all 8 features.

Let's denote mean of the output as $\mu_Y$. That is $\mu_y = \frac{1}{N}\sum y_i.$ Note then that for this prediction $\tilde{Y} (\tilde{y_i} = \mu_Y)$ Mean Square Error is equal to: 

$MSE(\tilde{Y})=\frac{1}{N}\sum ( y_i - \mu_Y)^2.$

In order to compare $MSE(\hat{Y})$ and variance of output, which we denote it as $D^2Y$. We take their difference and divide by the variance $D^2Y$. This is called $R^2$ score. This is then

$R^2 = \frac{D^2Y - MSE(\hat{Y})}{D^2Y} = 1- \frac{MSE(\hat{Y})}{D^2Y}$

Note that if $\hat{y}$ is nothing better than $\mu_Y$ then $R^2 = 0$. On the other side, if $MSE(\hat{Y}) = 0$, $R^2 =1$ meaning prefect prediction. Calculate $R^2$ for the above Q3 (a) and (b)? (hint: import r2_score function from the sklearn.metric)


## Q4. Splitting the data into a training and test set

In the experiments above, every model was fit to the *entire* data set and its mean squared error was evaluated on this same data set. This methodology would not, in general, yield accurate estimates of future error. In this specific case, however, the discrepancy might not be too bad because the data set is quite large relative to the number of features.

To investigate this further, we define a procedure **split_data** that partitions the data set into separate training and test sets. It is invoked as follows:

* `trainx, trainy, testx, testy = split_data(n_train)`

Here:
* `n_train` is the desired number of training points
* `trainx` and `trainy` are the training points and response values
* `testx` and `testy` are the test points and response values

The split is done randomly, but the random seed is fixed, and thus the same split is produced if the procedure is called repeatedly with the same `n_train` parameter.

In [8]:
def split_data(n_train):
    if (n_train < 0) or (n_train > 49999):
        print("Invalid number of training points")
        return
    np.random.seed(0)
    perm = np.random.permutation(49999)
    training_indices = perm[range(0,n_train)]
    test_indices = perm[range(n_train,49999)]
    trainx = x[training_indices,:]
    trainy = y[training_indices]
    testx = x[test_indices,:]
    testy = y[test_indices]
    return trainx, trainy, testx, testy

**<font color="magenta">For you to do:</font>** Using the **split_data** procedure to partition the data set, compute the training MSE and test MSE when fitting a regressor to *all* features, for the following training set sizes:
* `n_train = 20000`
* `n_train = 25000`
* `n_train = 30000`
* `n_train = 40000`

## Q5: Apply Min-Max feature scaling to normalize the data and caluclate the MSE and MAPE on features

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_scaled=scaler.fit_transform(x)

## Q6: Use test train split to divide the dataset in 70-30 and apply decision tree and Random forest using sklearn

Find the MSE and MAPE for both decision tree and random forest and compare it with linear regression and Plot the actual and predicted results of first 168 hours (weekly plot)

In [10]:
from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestRegressor

# Decision Tree Using sklearn

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42, shuffle=False)

# Random Forest using Sklearn

# Combine Results Plot

In [23]:
!pip install watermark 

In [36]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p wget,pandas,numpy,watermark,sklearn,matplotlib,keras

# date
print (" ")
%watermark -u -n -t -z 

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.29.0

wget      : 3.2
pandas    : 1.1.5
numpy     : 1.21.2
watermark : 2.3.1
sklearn   : 1.0.2
matplotlib: 3.3.4
keras     : 2.9.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 16
Architecture: 64bit

 
Last updated: Wed Aug 24 2022 11:43:14GMT Summer Time

